In [65]:
import pandas as pd
from sklearn import feature_extraction,linear_model,model_selection,naive_bayes

1. Загрузите датасет. Описание датасета можно посмотреть здесь.
2. Считайте датасет в Python (можете сразу грузить все в память, выборка небольшая), выясните, что используется в качестве разделителей и как проставляются метки классов.

In [15]:
data=pd.read_csv('SMSSpamCollection.txt',sep='\t',names=['class','text'])
data.head()

,class,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


3 Подготовьте для дальнейшей работы два списка: список текстов в порядке их следования в датасете и список соответствующих им меток классов. В качестве метки класса используйте 1 для спама и 0 для "не спама".

In [32]:
text=data.text
y=data['class'].apply(lambda x:1 if x=='spam' else 0 )

4 Используя sklearn.feature_extraction.text.CountVectorizer со стандартными настройками, получите из списка текстов матрицу признаков X.
5 Оцените качество классификации текстов с помощью LogisticRegression() с параметрами по умолчанию, используя sklearn.cross_validation.cross_val_score и посчитав среднее арифметическое качества на отдельных fold'ах. Установите random_state=2. Параметр cv задайте равным 10. В качестве метрики качества используйте f1-меру. Получившееся качество - один из ответов, которые потребуются при сдаче задания. Ответ округлить до 1 знака после запятой.

In [102]:
vectorizer=feature_extraction.text.CountVectorizer()
X=vectorizer.fit_transform(text)
lr=linear_model.LogisticRegression()
cv = model_selection.StratifiedKFold(n_splits=10, random_state=2) 
score=model_selection.cross_val_score(lr,X,y,cv=cv,scoring='f1')
CVScore=score.mean()
round(CVScore,1)

0.90000000000000002

6 А теперь обучите классификатор на всей выборке и спрогнозируйте с его помощью класс для следующих сообщений:

* "FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB"
* "FreeMsg: Txt: claim your reward of 3 hours talk time"
* "Have you visited the last lecture on physics?"
* "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$"
* "Only 99"

Прогнозы классификатора (0 - не спам, 1 - спам), записанные через пробел, будут ответом в одном из вопросов ниже.

In [73]:
Text_test=["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
"FreeMsg: Txt: claim your reward of 3 hours talk time",
"Have you visited the last lecture on physics?",
"Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
"Only 99$"]
X_test=vectorizer.transform(Text_test)

In [74]:
lr.fit(X,y)
ypred=lr.predict(X_test)
print (ypred)

[1 1 0 0 0]


7 Задайте в CountVectorizer параметр ngram_range=(2,2), затем ngram_range=(3,3), затем ngram_range=(1,3). Во всех трех случаях измерьте получившееся в кросс-валидации значение f1-меры, округлите до второго знака после точки, и выпишете результаты через пробел в том же порядке. В данном эксперименте мы пробовали добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. Обратите внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.

In [89]:
def get_ftr(ngram):
    vectorizer=feature_extraction.text.CountVectorizer(ngram_range=ngram)
    return vectorizer.fit_transform(text)
    
def cmp_models(clf):
    res=list()
    res.append(model_selection.cross_val_score(clf,get_ftr((2,2)),y,cv=cv,scoring='f1').mean())
    res.append(model_selection.cross_val_score(clf,get_ftr((3,3)),y,cv=cv,scoring='f1').mean())
    res.append(model_selection.cross_val_score(clf,get_ftr((1,3)),y,cv=cv,scoring='f1').mean())
    return [round(x,2) for x in res]

In [94]:
cmp_models(lr)

[0.81999999999999995, 0.72999999999999998, 0.93000000000000005]


8 Повторите аналогичный п.7 эксперимент, используя вместо логистической регрессии MultinomialNB(). Обратите внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.

По какой-то причине обучение наивного байесовского классификатора через Pipeline происходит с ошибкой. Чтобы получить правильный ответ, отдельно посчитайте частоты слов и обучите классификатор.

In [91]:
mnb=naive_bayes.MultinomialNB()
cmp_models(mnb)

[0.65000000000000002, 0.38, 0.89000000000000001]

9 Попробуйте использовать в логистической регрессии в качестве признаков Tf*idf из TfidfVectorizer на униграммах. Повысилось или понизилось качество на кросс-валидации по сравнению с CountVectorizer на униграммах? (напишите в файле с ответом 1, если повысилось, -1, если понизилось, и 0, если изменилось не более чем на 0.01). Обратите внимание, что результат перехода к tf*idf не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данные.

In [103]:
Tfidvec=feature_extraction.text.TfidfVectorizer()
X=Tfidvec.fit_transform(text)
score=model_selection.cross_val_score(lr,X,y,cv=cv,scoring='f1')
TfidScore=score.mean()
print ('CountVectorizer=',CVScore)
print ('TfidfVectorizer=',TfidScore)


CountVectorizer= 0.932640298361
TfidfVectorizer= 0.852859955417


In [104]:
print ("result=-1")

result=-1
